<img src="images/logo_city.png" align="right" width="20%">

# Data Generation Module Development

## 1. Goal

The module should be something like below:

In [1]:
import timeslice                    
import timeslice.source as source   
import timeslice.rule  as rule      
import timeslice.worker as worker
import timeslice.viz as viz # to be done

import torch

# connect to database source
taxi_db = source.DatabaseSource('cleaned_small_yellow_2017_05')

# set time split rule for dataset generation
time_rule = rule.TimeSlice(stp='2017-05-01 00:00:00', etp='2017-05-02 00:00:00', freq='10min')

# initialize worker
data_worker = worker.Worker(source=taxi_db, destin='data_test/monthly_data/may', rule=time_rule, viz=True)

# generate dataset
data_worker.generate()

2017-05-01 00:00:00 2017-05-01 00:10:00
2017-05-01 00:10:00 2017-05-01 00:20:00
2017-05-01 00:20:00 2017-05-01 00:30:00
2017-05-01 00:30:00 2017-05-01 00:40:00
2017-05-01 00:40:00 2017-05-01 00:50:00
2017-05-01 00:50:00 2017-05-01 01:00:00
2017-05-01 01:00:00 2017-05-01 01:10:00
2017-05-01 01:10:00 2017-05-01 01:20:00
2017-05-01 01:20:00 2017-05-01 01:30:00
2017-05-01 01:30:00 2017-05-01 01:40:00
2017-05-01 01:40:00 2017-05-01 01:50:00
2017-05-01 01:50:00 2017-05-01 02:00:00
2017-05-01 02:00:00 2017-05-01 02:10:00
2017-05-01 02:10:00 2017-05-01 02:20:00
2017-05-01 02:20:00 2017-05-01 02:30:00
2017-05-01 02:30:00 2017-05-01 02:40:00
2017-05-01 02:40:00 2017-05-01 02:50:00
2017-05-01 02:50:00 2017-05-01 03:00:00
2017-05-01 03:00:00 2017-05-01 03:10:00
2017-05-01 03:10:00 2017-05-01 03:20:00
2017-05-01 03:20:00 2017-05-01 03:30:00
2017-05-01 03:30:00 2017-05-01 03:40:00
2017-05-01 03:40:00 2017-05-01 03:50:00
2017-05-01 03:50:00 2017-05-01 04:00:00
2017-05-01 04:00:00 2017-05-01 04:10:00
